## Описание проекта
Для исследования применимости алгоритмов машинного обучения в задаче расчёта высоковакуумного турбомолекулярного насоса следует решить следующие задачи:

- Решить оптимизационную задачу нахождения минимального диаметра колеса ТМН;
- Сгенерировать данные и провести по ним EDA;
- Исследовать различные модели машинного обучения на предмет предсказания минимального диаметра;
- Выбрать лучшую предсказательную модель и уточнить диапазоны оптимальных величин.
- Если возможно, получить функциональную зависимость.


Импорт библиотек:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%config InlineBackend.figure_format = 'retina'

## Explorary Data Analys

Считываем данные из файла и создаём из них датафрейм:

In [3]:
with open('data.npy', 'rb') as f:
    data = np.load(f)

In [4]:
df = pd.DataFrame(data, columns=['c_1', 'alpha_rad', 'lambda', 'ab_1', 'F_min'])
df.head()

,c_1,alpha_rad,lambda,ab_1,F_min
0,0.2,0.174533,0.6,0.99,73.058050
1,0.2,0.261799,0.6,0.99,59.286271
2,0.2,0.349066,0.6,0.99,52.061780
3,0.2,0.436332,0.6,0.98,47.650602
4,0.2,0.523599,0.6,0.97,44.733609


Проверим количество уникальных значений в каждом столбце. Поскольку c_1 и alpha задавались в ходе оптимизации, то их количество определено заранее(55 и 8 соответственно).

In [5]:
df.nunique().sort_values()

lambda         1
alpha_rad      8
ab_1          13
c_1           55
F_min        440
dtype: int64

Находим, что в столбце lambda одно уникальное значение, которое примем оптимальным независимо от всех остальных параметров для первого колеса ТМН.

In [6]:
print(f'Оптимальное значение lambda = {df["lambda"].unique()}')

Оптимальное значение lambda = [0.6]


Для ab_1 имеется только 13 уникальных значений из 60 заданных изначально в сетке оптимизации. Для этого параметра оптимальные границы устанавливают от 0.6 до 1.2 . Для первого пакета колёс рекомендуется выбирать высокие значения и уменьшать их по мере изменения геометрии колёс. 

In [7]:
np.sort(df.ab_1.unique())

array([0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.01])

Это позволяет уточнить диапазон оптимальных отношений до $ab_1 \in \{0.88, 1.01\}$.

Выводы:

- Оптимальное значение $\lambda_{опт}=0.6$ независимо от геометрии колеса или откачиваемого газа.
- Оптимальный диапазон параметра $ab_{1опт} \in \{0.88, 1.01\}$.

## Построение моделей

Выводы:

## Общие выводы